# Тетрадка с подготовкой данных <code>raw_gis_houses_data</code>

<code>raw_gis_houses_data</code> используется в <code>apartment_houses_all_data.ipynb</code> для создания и заливки в БД таблички <code>apartment_houses_all_data</code>

Где взять сырые данные:
- нужно зайти по ссылке https://dom.gosuslugi.ru/#!/houses или найти поиском "Реестр объектов жилищного фонда"
- выбрать в Субъект РФ "Москва"
- нажать "Найти" внизу справа
- нажать "Скачать" вверху справа
- скачается папка/архив с несколькими csv, её и будем обрабатывать

In [1]:
import json
import pandas as pd
from postamats.utils import prepare_data
from postamats.utils.connections import DB
from postamats.utils.prepare_data import RAW_GIS_NAME

pd.set_option('display.max_columns', None)

In [2]:
DATA_FOLDER = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022'

In [3]:
prepared_gis = prepare_data.prepare_gis_houses_data(DATA_FOLDER)

prepare_dmr_houses_data started ... 
loading /Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022 ... 
files from path: ['/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_2.csv', '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_3.csv', '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_1.csv', '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_4.csv', '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_5.csv', '/Users/affernus/PROJECTS_DATA/hacks/p

100%|██████████| 68318/68318 [00:00<00:00, 757446.25it/s]


create_object_id started


100%|██████████| 68318/68318 [00:00<00:00, 201855.54it/s]


prepare_dmr_houses_data finished


In [4]:
prepared_gis

,address_gis,address_code_gis,guid_fias_gis,oktmo_code_gis,management_method_gis,management_ogrn_gis,management_kpp_gis,management_name_gis,house_type_gis,condition_gis,total_area_gis,living_area_gis,demolition_date_gis,kad_n_gis,guid_house_gis,object_id_gis
0,"108809, Москва г, п. Марушкинское, д. Марушкин...",f93a19c3-ffab-48b5-92c3-d10a49166a72,F93A19C3-FFAB-48B5-92C3-D10A49166A72,45949000,Не выбран,,,NaN,Жилой,Исправный,396.2,NaN,NaN,50:26:0170402:2106,dc41973a-8cdd-4298-b7d7-365cd5b96a09,c03236dfd8117f656eff7f061e9dc22c9034eef194602f...
1,"108809, Москва г, п. Марушкинское, д. Марушкин...",9e595cc4-fa15-4d22-bc17-380b691f47c1,9E595CC4-FA15-4D22-BC17-380B691F47C1,45949000,Не выбран,,,NaN,Жилой,Исправный,124.9,NaN,NaN,нет,407142e2-e27c-4973-b614-7cd939e5a037,26146600ec0d327435e3794dd10b7c7d5df5ebc24b7d53...
2,"108809, Москва г, п. Марушкинское, д. Марушкин...",281b5116-2cef-4364-a850-1bb102687bfa,35F6CD9F-509B-4C92-839F-B03F70DE53E5,45949000,Не выбран,,,NaN,Жилой,Исправный,209.3,NaN,NaN,77:18:0170406:50,66eabcec-e02b-408e-9629-d17935b14399,a27dab0f237685c6b51c23f5d1f9a54a04093294d83ba5...
3,"108809, Москва г, п. Марушкинское, д. Марушкин...",855583f2-ce42-4ad0-9c9a-cfca34288c7f,10860E59-E7D1-43C2-A19B-1856644E9B3C,45949000,Не выбран,,,NaN,Жилой,Исправный,145.6,NaN,NaN,77:18:0170406:388,505249be-5ff4-4f7f-bfdc-222aab4cae19,4889a6151ee13c2467aa6286deddeadca7ff66b308a382...
4,"108809, Москва г, п. Марушкинское, д. Марушкин...",75f9b36c-5b8e-4c81-9215-b5fc4d462c37,DBC3B094-D0E1-4FC3-A5F7-044B88FC8124,45949000,Не выбран,,,NaN,Жилой,Исправный,224.1,NaN,NaN,50:26:0170402:2279,dae214a7-d984-4c71-9c97-48215a5fa8bf,8da27c4a1c4aa64e8288e0583e4a7f0b4ea8357c74e5de...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68313,"129323, Москва г, проезд. Лазоревый, д. 6",689fd70f-3055-4465-8190-21ccac493d57,689FD70F-3055-4465-8190-21CCAC493D57,45361000,УО,1087746440396,771601001,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,6892.7,6892.0,NaN,77:02:0014011:1055,9d85374a-7c36-4f60-8cdb-43edc2300929,c971db8c354581e1856f6b49c958d35e9b686fa68a4f1e...
68314,"129323, Москва г, проезд. Лазоревый, д. 8",ca25b32a-eb6b-4d25-9dc9-de2d32e321f7,CA25B32A-EB6B-4D25-9DC9-DE2D32E321F7,45361000,УО,1087746440396,771601001,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,6892.9,6892.9,NaN,77:02:0014011:1056,d9596c10-41ba-4fb5-9da6-5fe8b03a8fb0,d0f80ed71a12ae9cca3884ccc470a3efc4476a2367917f...
68315,"129323, Москва г, проезд. Русанова, д. 11",e61605c6-439f-46ac-8feb-fe5faf478145,E61605C6-439F-46AC-8FEB-FE5FAF478145,45361000,УО,5167746441088,771601001,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,13456.2,9437.4,NaN,77:02:0014011:13860,a87c74af-1983-4b25-9ae4-529044cf0fb6,d4f521fac1c580b2589668b9888248031eccb8a5f4aafa...
68316,"129323, Москва г, проезд. Русанова, д. 25, кор...",f87cae10-8731-4507-9184-f28ad94ee09a,F87CAE10-8731-4507-9184-F28AD94EE09A,45361000,УО,1087746440396,771601001,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,29049.2,21213.0,NaN,77:02:0014011:13149,21357c11-18f4-4487-8281-4324d6df4ba9,490aedc64df445d46881d86e64ae514aac949daaf8640f...


In [5]:
CONFIG_PATH = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/db_config.json'
with open(CONFIG_PATH, mode='r') as db_file:
    db_config = json.load(db_file)

database = DB(db_config)
database.load_to_bd(prepared_gis, RAW_GIS_NAME, geo=False)